In [1]:
import findspark
findspark.init('/home/siddharth/spark-2.4.1-bin-hadoop2.7/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp_example').getOrCreate()

In [4]:
data = spark.read.csv('./Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection/SMSSpamCollection',
                      inferSchema = True,sep='\t')

In [5]:
data.show(truncate = False)

+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0 |_c1                                                                                                                                                                                                 |
+----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |
|ham |Ok lar... Joking wif u oni...                                                                                                                                                     

In [6]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','message')
data.show()

+-----+--------------------+
|class|             message|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import col,udf,length
from pyspark.sql.types import IntegerType

In [8]:
# defing user defined functin countLength

countLen = udf(lambda word:len(word),IntegerType())

In [9]:
# feature engg

data = data.withColumn('length',countLen(col('message')))
data.show()

+-----+--------------------+------+
|class|             message|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [10]:
data.groupBy('class').mean('length').show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformer

In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,StringIndexer,VectorAssembler

In [12]:
indexer = StringIndexer(inputCol='class',outputCol='class_to_num')
token = Tokenizer(inputCol='message',outputCol='token')
stop = StopWordsRemover(inputCol='token',outputCol='stop')
count = CountVectorizer(inputCol='stop',outputCol='count')
idf = IDF(inputCol='count',outputCol='idf')
assembler = VectorAssembler(inputCols=['idf','length'],outputCol='features')

In [13]:
from pyspark.ml import Pipeline

In [14]:
pipe = Pipeline(stages=[indexer,token,stop,count,idf,assembler])

In [15]:
clean_data = pipe.fit(data).transform(data)
clean_data = clean_data.select('class_to_num','features')

In [16]:
clean_data.head()

Row(class_to_num=0.0, features=SparseVector(13424, {7: 3.1126, 11: 3.2055, 31: 3.822, 61: 4.2072, 72: 4.322, 344: 5.4072, 625: 5.918, 731: 6.1411, 1409: 6.6801, 1598: 6.8343, 4485: 7.5274, 6440: 7.9329, 8092: 7.9329, 8838: 7.9329, 11344: 7.9329, 12979: 7.9329, 13423: 111.0}))

## Model fitting

In [17]:
from pyspark.ml.classification import NaiveBayes

In [18]:
train,test = clean_data.randomSplit([0.8,0.2])

In [19]:
nb_pred = NaiveBayes(labelCol='class_to_num').fit(train).transform(test)

In [20]:
nb_pred.show()

+------------+--------------------+--------------------+--------------------+----------+
|class_to_num|            features|       rawPrediction|         probability|prediction|
+------------+--------------------+--------------------+--------------------+----------+
|         0.0|(13424,[0,1,3,9,1...|[-573.89425146192...|[1.0,5.2042591450...|       0.0|
|         0.0|(13424,[0,1,7,8,1...|[-1156.0589778795...|[1.0,3.9470877807...|       0.0|
|         0.0|(13424,[0,1,14,78...|[-692.43401848881...|[1.0,1.9580540985...|       0.0|
|         0.0|(13424,[0,1,20,27...|[-986.63079674430...|[0.99999999884090...|       0.0|
|         0.0|(13424,[0,1,24,31...|[-339.07848446606...|[1.0,1.3817570316...|       0.0|
|         0.0|(13424,[0,1,27,88...|[-1554.2920816699...|[5.88881792697611...|       1.0|
|         0.0|(13424,[0,1,31,43...|[-337.16130898865...|[1.0,4.3836931882...|       0.0|
|         0.0|(13424,[0,1,72,10...|[-690.60084838314...|[1.0,3.1305876978...|       0.0|
|         0.0|(13424,

### Evaluation

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [22]:
acc_eval = MulticlassClassificationEvaluator(labelCol='class_to_num')

In [23]:
acc = acc_eval.evaluate(nb_pred)

In [24]:
acc

0.9122979430159032